In [1]:
t#bibliotecas
import unidecode
import os
from pyspark.sql.functions import udf,col,concat_ws, explode,length,abs, lower,lit
from pyspark.sql.types import StringType, IntegerType

#bibliotecas ML
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,RegexTokenizer, StopWordsRemover, CountVectorizer, NGram
from pyspark.ml.classification import NaiveBayes,RandomForestClassifier,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

#MLFlow
import mlflow
import mlflow.azureml
import mlflow.spark
import mlflow.mleap
import mlflow.pyfunc

#variables generales del notebook
pathBronce = 'dbfs:/mnt/lake/bronce/'
stopWordsCustomizados = ["amlo"] + StopWordsRemover.loadDefaultStopWords("spanish")
minTokenSize = 3
cantidadNGrams = 1

#establecemos el experimento de MlFlow
experimentName = '/Users/jugordon@microsoft.com/AI/clasificadorTwitter'
mlflow.set_experiment(experimentName)

In [2]:
tweetData = spark.read.csv(pathBronce+'twitterTraining/tweetsSentiment.csv',sep=";",header="True",encoding="ISO-8859-1")
#crear tabla temporal
tweetData.createOrReplaceTempView("tweetData")
display(tweetData)

usuario,texto,categoria
OMNIANOTICIAS,NACIONAL AMLO y Arturo Zaldívar firman iniciativa de reforma judicial,informativo
Mitzinas_cov,Hasta dónde nos va a llevar ElCacas Pues hasta donde nosotros lo permitamos pilas en 2021 La nota deja descubiertos algunos fraudes,negativo
maxalexanderr,AMLO EL SEGUNDO MEJOR PRESIDENTE DEL CONTINENTE lopezobrador_ fue calificado por encima de Justin Trudeau Cana,positivo
RAbujder,"Sería bueno conocer su diagnostico con respecto a Evo Morales , que lo expliquen bien",informativo
epigmenioibarra,"De arriba para abajo,como dice AMLO que hay que barrer con la corrupción,así hay que terminar con la impunidad: Ya cayó Emilio Lozoya falta Peña Nieto Ya cayo García Luna falta Felipe Calderón Siguen Fox y los suyos, gracias !",positivo
XmachoXx,Amoooooo AMLO Estaremos siempre contigo Te vamos a reelegir en el 2024 Conferencia Presidente,positivo
gemmcal,"No quiero que los feminicidios opaquen la rifa AMLO , como ven las tonterias de este señor",negativo
ExpresionCdna,zuritacarpio Por eso se la vive ladrando contra AMLO Es increíble que casí nadié vea el interés general antes que el individual,negativo
renesuarez26,lopezobrador_ Todo mi apoyo a nuestro presidente AMLO Dios le de larga vida y Salud Tenemos un gran Presidente Honesto,positivo
IsaiasLaraG,"AlexanAndretti Un panistas cuando le dices que AMLO es el mejor presidente del mundo , arriba amlo",positivo


In [3]:
%sql 
select categoria, count(*) as cantidad from tweetData group by categoria

categoria,cantidad
informativo,115
negativo,133
positivo,67


-sandbox

<div><img src="http://csablog.azurewebsites.net/wp-content/uploads/2019/05/procesoClasificadorTextos.jpg" style="height: 100px; margin: 20px"/></div>

In [5]:
#Quitamos acentos y convertimos a minusculas
def remove_accents(input_str):
  return unidecode.unidecode(input_str).lower()

remove_accents_udf = udf(remove_accents, StringType())
tweetData = tweetData.withColumn('textoSinAcentos',remove_accents_udf(tweetData.texto))

#Tokenización
regexTokenizer = RegexTokenizer(inputCol="textoSinAcentos", outputCol="words", pattern="\\W")
  #Establecemos token minimo
regexTokenizer.setMinTokenLength(minTokenSize)
#3. Quitar stop words
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stopWordsCustomizados)
#4. Convertimos a ngramas
ngram = NGram(n=cantidadNGrams, inputCol="filtered", outputCol="ngrams")
#5. Convertimos categoria de texto a numerica
label_stringIdx = StringIndexer(inputCol = "categoria", outputCol = "label")
#6 Vectorizar textos
countVectors = CountVectorizer(inputCol="ngrams", outputCol="features", vocabSize=200, minDF=2)

#Ejecutamos el pipeline completo
pipelineFeatures = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx])

pipelineFeaturesFit = pipelineFeatures.fit(tweetData)
dfTweets = pipelineFeaturesFit.transform(tweetData)
display(dfTweets.limit(20))

usuario,texto,categoria,textoSinAcentos,words,filtered,ngrams,features,label
OMNIANOTICIAS,NACIONAL AMLO y Arturo Zaldívar firman iniciativa de reforma judicial,informativo,nacional amlo y arturo zaldivar firman iniciativa de reforma judicial,"List(nacional, amlo, arturo, zaldivar, firman, iniciativa, reforma, judicial)","List(nacional, arturo, zaldivar, firman, iniciativa, reforma, judicial)","List(nacional, arturo, zaldivar, firman, iniciativa, reforma, judicial)","List(0, 200, List(48, 157, 196), List(1.0, 1.0, 1.0))",1.0
Mitzinas_cov,Hasta dónde nos va a llevar ElCacas Pues hasta donde nosotros lo permitamos pilas en 2021 La nota deja descubiertos algunos fraudes,negativo,hasta donde nos va a llevar elcacas pues hasta donde nosotros lo permitamos pilas en 2021 la nota deja descubiertos algunos fraudes,"List(hasta, donde, nos, llevar, elcacas, pues, hasta, donde, nosotros, permitamos, pilas, 2021, nota, deja, descubiertos, algunos, fraudes)","List(llevar, elcacas, pues, permitamos, pilas, 2021, nota, deja, descubiertos, fraudes)","List(llevar, elcacas, pues, permitamos, pilas, 2021, nota, deja, descubiertos, fraudes)","List(0, 200, List(129), List(1.0))",0.0
maxalexanderr,AMLO EL SEGUNDO MEJOR PRESIDENTE DEL CONTINENTE lopezobrador_ fue calificado por encima de Justin Trudeau Cana,positivo,amlo el segundo mejor presidente del continente lopezobrador_ fue calificado por encima de justin trudeau cana,"List(amlo, segundo, mejor, presidente, del, continente, lopezobrador_, fue, calificado, por, encima, justin, trudeau, cana)","List(segundo, mejor, presidente, continente, lopezobrador_, calificado, encima, justin, trudeau, cana)","List(segundo, mejor, presidente, continente, lopezobrador_, calificado, encima, justin, trudeau, cana)","List(0, 200, List(0, 1, 5, 171, 183), List(1.0, 1.0, 1.0, 1.0, 1.0))",2.0
RAbujder,"Sería bueno conocer su diagnostico con respecto a Evo Morales , que lo expliquen bien",informativo,"seria bueno conocer su diagnostico con respecto a evo morales , que lo expliquen bien","List(seria, bueno, conocer, diagnostico, con, respecto, evo, morales, que, expliquen, bien)","List(seria, bueno, conocer, diagnostico, respecto, evo, morales, expliquen, bien)","List(seria, bueno, conocer, diagnostico, respecto, evo, morales, expliquen, bien)","List(0, 200, List(25, 59), List(1.0, 1.0))",1.0
epigmenioibarra,"De arriba para abajo,como dice AMLO que hay que barrer con la corrupción,así hay que terminar con la impunidad: Ya cayó Emilio Lozoya falta Peña Nieto Ya cayo García Luna falta Felipe Calderón Siguen Fox y los suyos, gracias !",positivo,"de arriba para abajo,como dice amlo que hay que barrer con la corrupcion,asi hay que terminar con la impunidad: ya cayo emilio lozoya falta pena nieto ya cayo garcia luna falta felipe calderon siguen fox y los suyos, gracias !","List(arriba, para, abajo, como, dice, amlo, que, hay, que, barrer, con, corrupcion, asi, hay, que, terminar, con, impunidad, cayo, emilio, lozoya, falta, pena, nieto, cayo, garcia, luna, falta, felipe, calderon, siguen, fox, los, suyos, gracias)","List(arriba, abajo, dice, barrer, corrupcion, asi, terminar, impunidad, cayo, emilio, lozoya, falta, pena, nieto, cayo, garcia, luna, falta, felipe, calderon, siguen, fox, gracias)","List(arriba, abajo, dice, barrer, corrupcion, asi, terminar, impunidad, cayo, emilio, lozoya, falta, pena, nieto, cayo, garcia, luna, falta, felipe, calderon, siguen, fox, gracias)","List(0, 200, List(3, 4, 18, 23, 35, 82, 98, 108, 122, 172, 176), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0))",2.0
XmachoXx,Amoooooo AMLO Estaremos siempre contigo Te vamos a reelegir en el 2024 Conferencia Presidente,positivo,amoooooo amlo estaremos siempre contigo te vamos a reelegir en el 2024 conferencia presidente,"List(amoooooo, amlo, estaremos, siempre, contigo, vamos, reelegir, 2024, conferencia, presidente)","List(amoooooo, siempre, contigo, vamos, reelegir, 2024, conferencia, presidente)","List(amoooooo, siempre, contigo, vamo

In [6]:
print(stopWordsCustomizados)

['amlo', 'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'esté', 'estés', 'estemos', 'estéis', 'estén', 'estaré', 'estarás', 'estará', 'estaremos', 'estaréis', 'estarán', 'estaría', 'estarías', 'estaríamos', 'estaríais', 'estarían', 'estaba', 'estabas', 'estábamos', 'estabais', 'estaban', 'estuve', 'estuviste', 'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera', 'estuvieras', 'estuviéramos', 'estuvierais', 'estuvieran', 'estuviese', 'estuvieses', 'estuviésemos', 'estuvieseis', 'estuviesen', 'estando', 'estado', 'estada', 'estados', 'estadas', 'estad', 'he', 'has', 'ha', 'hemos', 'habéis', 'han', 'haya', 'hayas', 'hayamos', 'hayáis', 'hayan', 'habré', 'habrás', 'habrá', 'habremos', 'habréis', 'habrán', 'habría', 'habrías', 'habríamos', 'habríais', 'habrían', 'había', 'habías', 'habíamos', 'habíais', 'habían', 'hube', 'hubiste', 'hubo', 'hubimos', 'hubisteis', 'hubieron', 'hubiera', 'hubieras', 'hubiéramos', 'hubierais', 'hubieran', 'hubiese', 'hubieses', 'hubiésemos', 'hubieseis', 'hubiesen', 'habiendo', 'habido', 'habida', 'habidos', 'habidas', 'soy', 'eres', 'es', 'somos', 'sois', 'son', 'sea', 'seas', 'seamos', 'seáis', 'sean', 'seré', 'serás', 'será', 'seremos', 'seréis', 'serán', 'sería', 'serías', 'seríamos', 'seríais', 'serían', 'era', 'eras', 'éramos', 'erais', 'eran', 'fui', 'fuiste', 'fue', 'fuimos', 'fuisteis', 'fueron', 'fuera', 'fueras', 'fuéramos', 'fuerais', 'fueran', 'fuese', 'fueses', 'fuésemos', 'fueseis', 'fuesen', 'sintiendo', 'sentido', 'sentida', 'sentidos', 'sentidas', 'siente', 'sentid', 'tengo', 'tienes', 'tiene', 'tenemos', 'tenéis', 'tienen', 'tenga', 'tengas', 'tengamos', 'tengáis', 'tengan', 'tendré', 'tendrás', 'tendrá', 'tendremos', 'tendréis', 'tendrán', 'tendría', 'tendrías', 'tendríamos', 'tendríais', 'tendrían', 'tenía', 'tenías', 'teníamos', 'teníais', 'tenían', 'tuve', 'tuviste', 'tuvo', 'tuvimos', 'tuvisteis', 'tuvieron', 'tuviera', 'tuvieras', 'tuviéramos', 'tuvierais', 'tuvieran', 'tuviese', 'tuvieses', 'tuviésemos', 'tuvieseis', 'tuviesen', 'teniendo', 'tenido', 'tenida', 'tenidos', 'tenidas', 'tened']

In [7]:
# Dividiendo datos de entrenamiento y de prueba
(trainingData, testData) = tweetData.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 218
Test Dataset Count: 97

In [8]:
#algoritmo naive bayes
naiveBayesModel = NaiveBayes(smoothing=1.0)

#Pipeline completo
pipelineModel = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx,naiveBayesModel])

#entrenamiento
modelNB = pipelineModel.fit(trainingData)
#generar predicciones
predictions = modelNB.transform(testData)
predictions.select("texto","categoria","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 20)

#evaluando el modelo
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
precision = evaluator.evaluate(predictions)
print("Precisión del modelo: "+ str(precision))



+--------------------+-----------+--------------------+-----+----------+
 texto| categoria| probability|label|prediction|
+--------------------+-----------+--------------------+-----+----------+
 heraldodemexico ...| negativo|[0.95818804742357...| 0.0| 0.0|
 grupodimco Puebl...|informativo|[0.94025814602612...| 1.0| 0.0|
Afiliados Robos f...| negativo|[0.93714312676201...| 0.0| 0.0|
 JLozanoA SE ALEG...| negativo|[0.86424962406149...| 0.0| 0.0|
 amlover33 GlodeJ...| negativo|[0.85391671043634...| 0.0| 0.0|
que chingue a su ...| negativo|[0.84926541981954...| 0.0| 0.0|
 GlodeJo07 A mi m...| negativo|[0.82114779032682...| 0.0| 0.0|
 tatclouthier Pod...| negativo|[0.81160203197864...| 0.0| 0.0|
No se auto engañe...| negativo|[0.79804012440761...| 0.0| 0.0|
AMLO está dándole...| negativo|[0.76570307637755...| 0.0| 0.0|
Si AMLO es tenden...| negativo|[0.75787503913673...| 0.0| 0.0|
 El_Universal_Mx ...| negativo|[0.75692184688841...| 0.0| 0.0|
 AMLO llama fuchi...|informativo|[0.74180094675801...| 1.0| 0.0|
 Sergeluna_S Es u...|informativo|[0.72459302582059...| 1.0| 0.0|
@ElSoberano_mx @F...| negativo|[0.71895039895221...| 0.0| 0.0|
 BettyGordon_ Poe...| positivo|[0.71825811547612...| 2.0| 0.0|
Por supuesto que ...| negativo|[0.67583395164284...| 0.0| 0.0|
Después de cada m...|informativo|[0.67556032566837...| 1.0| 0.0|
 _JCarlosZ terela...| negativo|[0.66484870450267...| 0.0| 0.0|
 Fuchi caca la nu...|informativo|[0.65254363736444...| 1.0| 0.0|
+--------------------+-----------+--------------------+-----+----------+
only showing top 20 rows

Precisión del modelo: 0.6928616882402875

In [9]:
#usando random forest
randomForestModel = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 50, \
                            maxDepth = 10, \
                            maxBins = 32)

pipelineRandomForestModel = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx,randomForestModel])

# Train model with Training Data
rfModel = pipelineRandomForestModel.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.select("texto","categoria","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 20, truncate = 30)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
precision = evaluator.evaluate(predictions)
print("Precisión del modelo: "+ str(precision))


+------------------------------+-----------+------------------------------+-----+----------+
 texto| categoria| probability|label|prediction|
+------------------------------+-----------+------------------------------+-----+----------+
que chingue a su madre amlo...| negativo|[0.6431274091506439,0.27321...| 0.0| 0.0|
 GlodeJo07 A mi me da vergü...| negativo|[0.6394024170866106,0.27772...| 0.0| 0.0|
No se auto engañen amigos e...| negativo|[0.5855827673231918,0.30628...| 0.0| 0.0|
 MxHija amigodeamlo0101 Kar...| negativo|[0.5799194944258763,0.32329...| 0.0| 0.0|
Afiliados Robos fraudes y n...| negativo|[0.5155814719704375,0.28981...| 0.0| 0.0|
@ElSoberano_mx @FelipeCalde...| negativo|[0.504463778298138,0.378508...| 0.0| 0.0|
 JLozanoA SE ALEGRA DE LO Q...| negativo|[0.5003766015646216,0.38361...| 0.0| 0.0|
Si AMLO es tendencia por al...| negativo|[0.4963193570234099,0.38186...| 0.0| 0.0|
 heraldodemexico MexLibre_ ...| negativo|[0.4765758437041718,0.38098...| 0.0| 0.0|
@ChayraIrma @ubastien Yo cr...| negativo|[0.47516136001532766,0.4048...| 0.0| 0.0|
 beltrandelrio AMLO Los uso...| negativo|[0.471863485578647,0.394448...| 0.0| 0.0|
 amlover33 GlodeJo07 Me cho...| negativo|[0.46982237157316986,0.4025...| 0.0| 0.0|
 Sergeluna_S Es un echo que...|informativo|[0.46910604338858064,0.3913...| 1.0| 0.0|
Que le quiten los guardaesp...|informativo|[0.4664202674204187,0.40517...| 1.0| 0.0|
 tatclouthier Podríamos dec...| negativo|[0.46316882484449784,0.4079...| 0.0| 0.0|
 grupodimco Puebla y todo M...|informativo|[0.4609124733118638,0.39064...| 1.0| 0.0|
Los discursos de AMLO son c...| negativo|[0.460523262832877,0.408145...| 0.0| 0.0|
 Fuchi caca la nueva forma ...|informativo|[0.4605129108660034,0.40745...| 1.0| 0.0|
@FelipeCalderon OjalÃ¡ te q...| negativo|[0.4596371225077774,0.41512...| 0.0| 0.0|
 AMLO llama fuchi caca a lo...|informativo|[0.45920494040509163,0.3826...| 1.0| 0.0|
+------------------------------+-----------+------------------------------+-----+----------+
only showing top 20 rows

Precisión del modelo: 0.567027842064661

In [10]:
with mlflow.start_run(run_name="Clasificador NaiveBayes cross validator") as run:

  #algoritmo naive bayes
  naiveBayesModel = NaiveBayes()

  #Pipeline completo
  pipelineNaiveBayes = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx,naiveBayesModel])
  
  # Create ParamGrid for Cross Validation
  paramGrid = ParamGridBuilder().addGrid(naiveBayesModel.smoothing, [0.2, 0.4, 0.6, 0.8, 1.0,1.5]).build()
  
  # Create 5-fold CrossValidator
  cvEvaluator = MulticlassClassificationEvaluator(predictionCol="prediction",metricName="weightedPrecision")

  cv_nb = CrossValidator(estimator=pipelineNaiveBayes, estimatorParamMaps=paramGrid, evaluator=cvEvaluator,numFolds=3)

  nbModel = cv_nb.fit(trainingData)
  #mlflow.spark.log_model(nbModel,"modeloNB")
  
  # Evaluate best model
  predictions = nbModel.transform(testData)
  test_metric  = evaluator.evaluate(predictions)
  
  #Resgistro de metricas en MlFlow 
  mlflow.log_metric('test_nb_' + evaluator.getMetricName(), test_metric) # Logs additional metrics
  
  #Registro de modelo
  #mlflow.mleap.log_model(spark_model=nbModel.bestModel, sample_input=testData, artifact_path='best-modelNB',registered_model_name="Clasificador NaiveBayes Twitter")
  

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [11]:
with mlflow.start_run(run_name="Clasificador Random Forest cross validator") as run:

  randomForestModel = RandomForestClassifier()

  pipelineRandomForestModel = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram,countVectors,label_stringIdx,randomForestModel])

  
  # Create ParamGrid for Cross Validation
  paramGrid = (ParamGridBuilder()
               .addGrid(randomForestModel.numTrees, [10, 100, 200]) # numTrees
               .addGrid(randomForestModel.maxDepth, [5, 10, 15, 20]) # maxDepth
               .addGrid(randomForestModel.maxBins, [16, 32]) #Number of maxBins
               .build())
  # Create 5-fold CrossValidator
  evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",metricName="weightedPrecision")
  evaluatorMSE = MulticlassClassificationEvaluator(predictionCol="prediction",metricName="mse")

  cv = CrossValidator(estimator=pipelineRandomForestModel, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

  cvModel = cv.fit(trainingData)
  
  # Evaluate best model
  #bestModelRF = cvModel.bestModel
  predictions = cvModel.transform(testData)
  test_metric  = evaluator.evaluate(predictions)
  
  #Resgistro de metricas en MlFlow 
  mlflow.log_metric('test_' + evaluator.getMetricName(), test_metric) # Logs additional metrics
  mlflow.mleap.log_model(spark_model=cvModel.bestModel, sample_input=testData, artifact_path='best-model') # 
  
  #Registro de modelo
  mlflow.mleap.log_model(spark_model=nbModel.bestModel, sample_input=testData, artifact_path='best-modelNB',registered_model_name="Clasificador RandomForest Twitter")

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
Successfully registered model 'Clasificador RandomForest Twitter'.
Created version '1' of model ''.

In [12]:

model_name = "twitterSentimentClassificator.mml" # 
#model_name_dbfs = os.path.join("/dbfs", model_name)
model_name_dbfs = "dbfs:/mnt/models/" + model_name

cvModel.bestModel.write().overwrite().save(model_name)

print("copy model from dbfs to local")
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_name, model_local, True)

copy model from dbfs to local
Out[12]: True